# Simulation Optimsiation using DES model of homeless response system

In [1]:
import datetime
print('Current version of this notebook updated ' + str(datetime.date.today()))

Current version of this notebook updated 2023-10-25


## Ranking & Selection

First we employ a Ranking & Selection algorithm based on the Kim & Nelson procedure. Details of this procedure can be found in section 9.3.2 (page 247) of 'Foundation and Methods of Stochastic Simulation' - Edition 2 (2021). The Python code for this procedure is found in the ranking_and_selection.py file in the GitHub repository. Below we import this module, and some others

In [2]:
# modules from this repository
import ranking_and_selection as rs
import simulation_model as sim
import queueing_model as qm

# external packages
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd

### Testing using Inventory System

In order to test our KN procedure, we test it on a simulation model which has already been analysed using the KN algorithm in STOR-606 module - this is an $(s,S)$ inventory system where stock is replenished to a level of $S$ when it reaches $s$. 

In [3]:
solutions = [i for i in range(1600)]
k=np.array([i for i in range(1600)])

def simulate(solution):
    # one replication of simulating the cost of the inventory policy
    out=rs.InventorySystem(solution)[0]
    return out

In [4]:
random.seed(1)
opt_sols = []
for i in range(5):
    spc = rs.SolutionSpace(solutions)
    spc.optimise_rs(0.05, 50, 1, simulate, False)
    s,S = rs.get_sS_system(k[spc.active][0])
    opt_sols.append((s,S))

In [5]:
print('(s,S) for the optimal solution found at each iteration of the algorithm')
print(opt_sols)

(s,S) for the optimal solution found at each iteration of the algorithm
[(15, 51), (16, 52), (15, 51), (12, 48), (9, 48)]


The above illustrates that this KN algorithm can return different solutions when it is run at different times (i.e. with different starting seeds) - this is likely due to the difference between the true best and other good solutions being less than the 'delta' indiffference zone parameter used when running the algorithm above. 

### Developing a discrete solution space for the homeless response system

In [6]:
# solution space constraints
build_rate_options = {'housing' : [12, 24], 'shelter' : [12,24]}
annual_budget = 36
accommodation_budgets = {'housing' : 96, 'shelter' : 96}
simulation_length = 5

# simulation options
number_reps = 1
initial_build_time = 63/365 # 9 weeks in years
end_of_simulation = 5 + initial_build_time - 0.25 # in years
initial_demand = 120
initial_capacity = {'housing' : 40, 'shelter' : 15}
arrival_rates = [35.0400, 42.0048, 46.2528, 46.2528, 41.6100] # in 1/year. One constant rate per year.
service_mean = {'housing' : (1/52)*(0+300+400)/3, 'shelter' : 0.0} # in years

# adjust arrival rates to include re-entries
reentry_rate = 0.17 # the proportion of those leaving accommodation which re-enter the system some time later
arrival_rate_reentries = (initial_capacity['housing']*reentry_rate)/service_mean['housing'] # assuming re-entries from the initial number of servers
arrival_rates = [i+arrival_rate_reentries for i in arrival_rates]#
time_btwn_changes_in_build_rate = (6*63)/365 # in years
time_btwn_building = 63/365 # in years. 63/365 years = 9 weeks.
reentry_rate = 0 # set this to zero now we have accounted for re-entries using an uplift to arrival rate

# additional params for analytical model
max_in_system = 1000
num_annual_buildpoints = 6
build_frequency_weeks = 9
d = 1 # days

# geneate solution space
sols = rs.generate_solution_space(build_rate_options, annual_budget, accommodation_budgets, simulation_length)

Below we initialise a solution space object with the solutions we have generated

In [7]:
spc = rs.SolutionSpace(sols)

We next set a seed and then look for an optimal solution using the KN algorithm. A line of text is printed below whenever solutions are removed from the candidate list by the algorithm. 

In [ ]:
random.seed(1)
spc.optimise_rs(0.05, 100, 0.74, sim.simulate_as_is, True)

starting routine at time  2023-10-25 16:15:12.451282
done init reps at time  2023-10-25 16:38:30.814277
start iteration 102 with 109 active solutions out of initial 221 at time 2023-10-25 16:38:45.308388
start iteration 116 with 108 active solutions out of initial 221 at time 2023-10-25 16:40:24.704878
start iteration 126 with 107 active solutions out of initial 221 at time 2023-10-25 16:41:34.720320
start iteration 148 with 106 active solutions out of initial 221 at time 2023-10-25 16:44:05.995279
start iteration 158 with 105 active solutions out of initial 221 at time 2023-10-25 16:45:14.033535
start iteration 166 with 104 active solutions out of initial 221 at time 2023-10-25 16:46:07.870600
start iteration 184 with 103 active solutions out of initial 221 at time 2023-10-25 16:48:07.433706
start iteration 188 with 102 active solutions out of initial 221 at time 2023-10-25 16:48:33.632847
start iteration 198 with 101 active solutions out of initial 221 at time 2023-10-25 16:49:38.856

The details of the optimal solution are given below, followed by the following 20 solutions in decreasing order of the iteration number at which the KN algorithm removed them from the candidate list. 

In [ ]:
# create dataframe
my_list = []
for i in range(len(sols)):
    my_dict = {}
    for index, element in enumerate(sols[i]['housing']):
        my_dict[index] = element
    for index, element in enumerate(sols[i]['shelter']):
        my_dict[index+5] = element
    my_dict[10] = spc.eliminate[i]
    my_dict[11] = np.mean(spc.costs[i])
    my_dict[12] = np.var(spc.costs[i])
    my_list.append(my_dict)
df_sim = pd.DataFrame.from_dict(my_list)
df_sim.columns = ['h1', 'h2', 'h3', 'h4', 'h5', 'sh1', 'sh2', 'sh3', 'sh4', 'sh5', 'iter_elim', 'sim_mean', 'sim_var']

In [ ]:
# model analytically
outputs = []
for s in range(len(sols)):
    q = qm.queue(arrival_rates, service_mean['housing'], initial_capacity['housing'], initial_capacity['shelter'], [int(sols[s]['housing'][i]/num_annual_buildpoints) for i in range(len(sols[s]['housing']))], [int(sols[s]['shelter'][i]/num_annual_buildpoints) for i in range(len(sols[s]['shelter']))], initial_demand, max_in_system, num_annual_buildpoints, build_frequency_weeks)
    q.model_dynamics(simulation_length-0.25, d)
    outputs.append(q.num_unsheltered_avg)
    print('done ' + str(s) + ' at time ' + str(datetime.datetime.now()))

### Build rates of 15 or 30

#### Histogram of results

In [ ]:
plt.hist(outputs,200)
plt.show()

#### Dataframe of results

In [ ]:
my_list = []
for i in range(len(sols)):
    my_dict = {}
    for index, element in enumerate(sols[i]['housing']):
        my_dict[index] = element
    for index, element in enumerate(sols[i]['shelter']):
        my_dict[index+5] = element
    my_dict[10] = outputs[i]
    my_list.append(my_dict)
df_analytic = pd.DataFrame.from_dict(my_list)
df_analytic.columns = ['h1', 'h2', 'h3', 'h4', 'h5', 'sh1', 'sh2', 'sh3', 'sh4', 'sh5', 'c']

In [ ]:
df_concat = df_analytic.sort_values(by='c',ascending=True).reset_index(drop=True).merge(df_sim, on = ['h1', 'h2', 'h3', 'h4', 'h5', 'sh1', 'sh2', 'sh3', 'sh4', 'sh5'])
df_concat.style.apply(lambda x: ["background: pink" if v == 24 else "" for v in x], axis = 1)

#### Looking at improvements

In [ ]:
low_val = 12
dims_all = ['h1','h2','h3','h4','h5','sh1','sh2','sh3','sh4','sh5']
list_imprv = [] # data on improvements
for dim in dims_all:
    for i in df_analytic[df_analytic[dim]==low_val].index.tolist():
        value_base = df_analytic.iloc[i]['c']
        conditions = dim + '''!=''' + str(low_val)
        for dim_compare in [dims_all[j] for j in range(len(dims_all)) if dims_all[j]!=dim]:
            conditions = conditions + ''' & ''' + dim_compare + '==' + str(df_analytic.iloc[i][dim_compare])
        df_analytic_with_conditions = df_analytic.query(conditions).reset_index(drop=True)
        if len(df_analytic_with_conditions) > 0:
            value_improve = df_analytic_with_conditions['c'][0]
            improvement = value_base - value_improve
            improvement = {'dim': dim,
                           'imprv': improvement}
            list_imprv.append(improvement)
        
df_imprv = pd.DataFrame.from_dict(list_imprv)

In [ ]:
df_imprv_summary = df_imprv.groupby('dim').mean().reset_index()
df_imprv_summary

#### Checking the analytical and simulation models agree on results

In [ ]:
q = qm.queue(arrival_rates, service_mean['housing'], initial_capacity['housing'], initial_capacity['shelter'], [4,4,4,2,2], [2,2,2,4,4], initial_demand, max_in_system, num_annual_buildpoints, build_frequency_weeks)
q.model_dynamics(simulation_length-0.25, d)
q.num_unsheltered_avg

#### Build rates of 25 or 50

When the build rate options are 25 or 50, then in year 1, we either build 75 units of accommodation, which immediately clears the queue, or we build 50 units which doesn't immediately clear the queue, but at the next year does. 

In [ ]:
build_rate_options1 = {'housing' : [24, 48], 'shelter' : [24, 48]}
annual_budget1 = 72
accommodation_budgets1 = {'housing' : 192, 'shelter' : 192}
simulation_length1 = 5

sols1 = rs.generate_solution_space(build_rate_options1, annual_budget1, accommodation_budgets1, simulation_length1)

# model analytically
outputs1 = []
for s in range(len(sols1)):
    q = qm.queue(arrival_rates, service_mean['housing'], initial_capacity['housing'], initial_capacity['shelter'], [int(sols1[s]['housing'][i]/num_annual_buildpoints) for i in range(len(sols1[s]['housing']))], [int(sols1[s]['shelter'][i]/num_annual_buildpoints) for i in range(len(sols1[s]['shelter']))], initial_demand, max_in_system, num_annual_buildpoints, build_frequency_weeks)
    q.model_dynamics(simulation_length1-0.25, d)
    outputs1.append(q.num_unsheltered_avg)
    print('done ' + str(s) + ' at time ' + str(datetime.datetime.now()))

In [ ]:
plt.hist(outputs1,200)
plt.show()